In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
# pd.set_option('display.max_colwidth', None)

In [4]:
pd.reset_option('display.max_colwidth')

In [5]:
master_data = {
    "gratka_2023_master" : "../original_data_2023/gratka/ALL_DATA/gdansk_gratka_master_data.json",
}

for key, value in master_data.items():
    master_data[key] = pd.read_json(value)  

gratka_df = master_data["gratka_2023_master"]

In [6]:
df = gratka_df

In [7]:
df.shape

(1647, 31)

In [8]:
df.head()

,link,rent,description,address,title,additional_info,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,...,Materiał budynku,Liczba miejsc parkingowych,Czy mieszkanie ma łazienkę?,Możliwy kontakt w języku,"Opłaty (czynsz administracyjny, media)",Stan instalacji,Stan łazienki,Okna,Głośność,Usytuowanie względem stron świata
0,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 500,"Inwestycja BRABANK zachwyca lokalizacją, archi...",Gdańsk pomorskie,Mieszkanie Gdańsk ul. Stara Stocznia,[],"Gdańsk, ...","68,69 m2",3.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 450,"homfi ma przyjemność zaprezentować jasne, prze...",Gdańsk Brzeźno pomorskie,Mieszkanie Gdańsk Brzeźno ul. Dunikowskiego,[],"Gdańsk, ...",47 m2,2.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 800,Zamieszkaj w stylowej kamienicy. Zapraszamy do...,Gdańsk Stare Miasto pomorskie,Mieszkanie Gdańsk Stare Miasto ul. Tobiasza,[],"Gdańsk, ...","100,40 m2",4.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://gratka.pl/nieruchomosci/mieszkanie-gda...,5 400,Lokal 125 m/2 w Gdańsk Oliwa u zbiegu ul Al. G...,Gdańsk Oliwa pomorskie,Mieszkanie Gdańsk Oliwa ul. Grunwaldzka,[],"Gdańsk, ...",125 m2,4.0,2,...,cegła,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 500,--==Rezerwacja==-- --==Rezerwacja==-- --==Rese...,Gdańsk Piecki-Migowo pomorskie,Mieszkanie Gdańsk Piecki-Migowo ul. Myśliwska,[],"Gdańsk, ...",40 m2,2.0,3,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# df['Lokalizacja'].value_counts()

# df['Lokalizacja'].str.replace("Gdańsk,", "").str.replace("pomorskie", "").str.strip().value_counts()

# regions_dict = {}  # Initialize the dictionary outside the loop

# for number in range(otodom_gda_df.shape[0]):
#     address = otodom_gda_df['address'].str.split(",")[number]
#     if address and len(address) > 1:
#         region = address[1].strip()
#         if region not in regions_dict:
#             regions_dict[region] = 1
#         else:
#             regions_dict[region] += 1

# import pprint
# pprint.pprint(regions_dict)

# df['address'].str.split(",")[8][1].strip()

# df['address']

# # Assuming you have a DataFrame named 'df' and you want to check a specific column named 'your_column_name'
# target_string = 'negocjacji'
# column_name = 'rent'

# # Create a boolean mask to check if each row in the specified column contains the target string
# mask = olx_gda_df[column_name].str.contains(target_string, na=False)

# # Count the number of True values in the mask (i.e., the number of rows containing the target string)
# count = mask.sum()

# print(f"The string '{target_string}' appears in {count} rows of the column '{column_name}'.")

# df['location'].value_counts()

In [10]:
gratka_gda_districts_mapping = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Przymorze Wielkie": "Przymorze",
    "Brzeźno": "Brzezno",
    "Przymorze Małe": "Przymorze",
    "Wrzeszcz": "Wrzeszcz",
    "Żabianka - Wejhera - Jelitkowo - Tysiąclecia": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Chełm z dzielnicą Gdańsk Południe": "Chelm - Lostowice - Poludnie",
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Ujeścisko - Łostowice": "Chelm - Lostowice - Poludnie",
    "Letnica": "Letnica",
    "Piecki-Migowo": "Piecki Migowo - Suchanino - Morena",
    "Zaspa Młyniec": "Zaspa",
    "Orunia - Św. Wojciech - Lipce": "Orunia",
    "Zaspa Rozstaje": "Zaspa",
    "Siedlce": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Stogi z Przeróbką": "Przerobka - Stogi - Nowy Port",
    "Oliwa": "Oliwa - Strzyza",
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Wzgórze Mickiewicza": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Brętowo": "Piecki Migowo - Suchanino - Morena",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Strzyża": "Oliwa - Strzyza",
    "Młyniska": "Siedlce - Aniolki - Mlyniska - Mickiewicza"
}

gratka_rooms_mapping = {
    3: 3,
    2: 2,
    1: 1,
    4: 4,
    5: 4,
    6: 4,
    7: 4
}

gratka_private_seller_mapping = {
    1.0: True,
    np.nan: False
}

gratka_furnished_mapping = {
    "Tak": True,
}


gratka_level_mapping = {
    'parter': 0,
    '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10,
    '11': 11,
    '12': 11,
    '13': 11,
    '14': 11,
    '15': 11,
    '16': 11,
    '17': 11,
    'poddasze': 11   
}

gratka_building_type_mapping = {
    "blok": "unit",
    "apartamentowiec": "apartment_building",
    "kamienica": "tenement",
    "dom wielorodzinny/szeregowiec": "house",
    "loft": "unit"
}

In [11]:
dirty_data = df.copy()

In [12]:
dirty_data["Typ zabudowy"].value_counts()

Typ zabudowy
apartamentowiec                  535
blok                             433
kamienica                        127
dom wielorodzinny/szeregowiec      8
loft                               1
Name: count, dtype: int64

In [13]:
dirty_data["Piętro"].value_counts()

Piętro
1           360
2           342
3           222
parter      220
4           176
5           103
6            49
7            40
9            17
11           14
8            14
10           14
12           14
16           10
17           10
13            7
15            6
14            5
poddasze      1
Name: count, dtype: int64

In [14]:
df = dirty_data.copy()
df.isna().sum()

link                                         0
rent                                         0
description                                  0
address                                      0
title                                        0
additional_info                              0
Lokalizacja                                  0
Powierzchnia w m2                            1
Liczba pokoi                                 4
Piętro                                      23
Rok budowy                                 433
Typ zabudowy                               543
Liczba pięter w budynku                     84
Dostępność od                              994
Numer referencyjny                          95
Udogodnienia                               441
Zaktualizowane                               0
Dodane                                       0
Stan                                      1341
Miejsce parkingowe                        1366
Forma kuchni                              1254
Materiał budy

In [15]:
df.columns

Index(['link', 'rent', 'description', 'address', 'title', 'additional_info',
       'Lokalizacja', 'Powierzchnia w m2', 'Liczba pokoi', 'Piętro',
       'Rok budowy', 'Typ zabudowy', 'Liczba pięter w budynku',
       'Dostępność od', 'Numer referencyjny', 'Udogodnienia', 'Zaktualizowane',
       'Dodane', 'Stan', 'Miejsce parkingowe', 'Forma kuchni',
       'Materiał budynku', 'Liczba miejsc parkingowych',
       'Czy mieszkanie ma łazienkę?', 'Możliwy kontakt w języku',
       'Opłaty (czynsz administracyjny, media)', 'Stan instalacji',
       'Stan łazienki', 'Okna', 'Głośność',
       'Usytuowanie względem stron świata'],
      dtype='object')

In [16]:
df.head()

,link,rent,description,address,title,additional_info,Lokalizacja,Powierzchnia w m2,Liczba pokoi,Piętro,...,Materiał budynku,Liczba miejsc parkingowych,Czy mieszkanie ma łazienkę?,Możliwy kontakt w języku,"Opłaty (czynsz administracyjny, media)",Stan instalacji,Stan łazienki,Okna,Głośność,Usytuowanie względem stron świata
0,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 500,"Inwestycja BRABANK zachwyca lokalizacją, archi...",Gdańsk pomorskie,Mieszkanie Gdańsk ul. Stara Stocznia,[],"Gdańsk, ...","68,69 m2",3.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 450,"homfi ma przyjemność zaprezentować jasne, prze...",Gdańsk Brzeźno pomorskie,Mieszkanie Gdańsk Brzeźno ul. Dunikowskiego,[],"Gdańsk, ...",47 m2,2.0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://gratka.pl/nieruchomosci/mieszkanie-gda...,6 800,Zamieszkaj w stylowej kamienicy. Zapraszamy do...,Gdańsk Stare Miasto pomorskie,Mieszkanie Gdańsk Stare Miasto ul. Tobiasza,[],"Gdańsk, ...","100,40 m2",4.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://gratka.pl/nieruchomosci/mieszkanie-gda...,5 400,Lokal 125 m/2 w Gdańsk Oliwa u zbiegu ul Al. G...,Gdańsk Oliwa pomorskie,Mieszkanie Gdańsk Oliwa ul. Grunwaldzka,[],"Gdańsk, ...",125 m2,4.0,2,...,cegła,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://gratka.pl/nieruchomosci/mieszkanie-gda...,2 500,--==Rezerwacja==-- --==Rezerwacja==-- --==Rese...,Gdańsk Piecki-Migowo pomorskie,Mieszkanie Gdańsk Piecki-Migowo ul. Myśliwska,[],"Gdańsk, ...",40 m2,2.0,3,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df = dirty_data.copy()

def convert_numeric_to_float(convertable_string):
    return float(convertable_string.replace("zł", "").replace(" ", "").replace("\ndonegocjacji", ""))

def clear_gratka(df: pd.DataFrame, city = "gda"):
    df.dropna(subset=['Liczba pokoi', 'Powierzchnia w m2'], inplace=True)
    
    df['Piętro'] = df['Piętro'].fillna(1001)
    df["Typ zabudowy"] = df["Typ zabudowy"].fillna("Unknown")
    
    df.drop(
        labels=[
            'Usytuowanie względem stron świata',
            'Stan',
            'Miejsce parkingowe',
            'Forma kuchni',
            'Materiał budynku',
            'Liczba miejsc parkingowych',
            'Czy mieszkanie ma łazienkę?',
            'Możliwy kontakt w języku',
            'Opłaty (czynsz administracyjny, media)',
            'Stan instalacji',
            'Stan łazienki',
            'Okna',
            'Dostępność od',
            'Zaktualizowane',
            'Głośność',
            'link',
            'description',
            'Numer referencyjny',
            'Dodane'
        ],
        inplace= True,
        axis= 1
    )
    
#     is_negotiable_list = []
#     is_furnished_list = []
    
#     for index, row in df.iterrows():

#         # negotiable
#         if 'do negocjacji' in row['rent']:
#             is_negotiable_list.append(True)
#         else:
#             is_negotiable_list.append(False)

#         # furnished
#         if 'Tak' in row['Umeblowane']:
#             is_furnished_list.append(True)
#         else:
#             is_furnished_list.append(False)

#     # negotiable apply
#     df['negotiable'] = is_negotiable_list

    # building_type
    df = df.rename(mapper={"Typ zabudowy": "building_type"}, axis=1)
    df["building_type"] = df["building_type"].str.replace(" ", "").str.lower()
    df["building_type"] = df["building_type"].map(gratka_building_type_mapping)
    
#     # furnished apply
#     df = df.rename(mapper={"Umeblowane": "furnished"}, axis=1)
#     df['furnished'] = is_furnished_list
    
#     # private_seller
#     df = df.rename(mapper={"Prywatne": "private_seller"}, axis=1)
#     df['private_seller'] = df['private_seller'].map(gratka_private_seller_mapping)
    
#     # negotiable apply
#     df['negotiable'] = is_negotiable_list
    
    # surface
    df = df.rename(mapper={"Powierzchnia w m2": "surface"}, axis=1)
    df['surface'] = df['surface'].str.replace("m²", "").str.replace("m2", "").str.replace(" ", "").str.replace(",", ".").astype(float)

    # level 
    df = df.rename(mapper={"Piętro": "level"}, axis=1)
    df['level'] = df['level'].map(gratka_level_mapping)
    
    # rooms 
    df = df.rename(mapper={"Liczba pokoi": "rooms"}, axis=1)
    df['rooms'] = df['rooms'].map(gratka_rooms_mapping)
    df['rooms'] = df['rooms'].astype(int)
    
    # rent
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

#     # rent_extra
#     df = df.rename(mapper={"Czynsz (dodatkowo)": "rent_extra"}, axis=1)
#     df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

#     # location
#     if city.lower() == "gda":
#         df['location'] = df['location'].map(gratka_gda_districts_mapping)
#     elif city.lower() == "krk":
#         pass
#         # df['location'] = df['location'].map(gratka_krk_districts_mapping)

    return df

In [18]:
df_cleared = clear_gratka(df)

In [19]:
df_cleared.isna().sum()

rent                         0
address                      0
title                        0
additional_info              0
Lokalizacja                  0
surface                      0
rooms                        0
level                       19
Rok budowy                 429
building_type              547
Liczba pięter w budynku     80
Udogodnienia               437
dtype: int64

In [20]:
df_cleared["rooms"].unique()

array([3, 2, 4, 1])

In [21]:
df_cleared["building_type"].unique()

array(['apartment_building', 'unit', 'tenement', nan], dtype=object)

In [22]:
df_cleared.columns

Index(['rent', 'address', 'title', 'additional_info', 'Lokalizacja', 'surface',
       'rooms', 'level', 'Rok budowy', 'building_type',
       'Liczba pięter w budynku', 'Udogodnienia'],
      dtype='object')

In [23]:
df_cleared["building_type"].value_counts()

building_type
apartment_building    535
unit                  433
tenement              127
Name: count, dtype: int64

In [24]:
df_cleared.head()

,rent,address,title,additional_info,Lokalizacja,surface,rooms,level,Rok budowy,building_type,Liczba pięter w budynku,Udogodnienia
0,6500.0,Gdańsk pomorskie,Mieszkanie Gdańsk ul. Stara Stocznia,[],"Gdańsk, ...",68.69,3,4.0,2018.0,apartment_building,6,"winda, teren zamknięty, domofon"
1,2450.0,Gdańsk Brzeźno pomorskie,Mieszkanie Gdańsk Brzeźno ul. Dunikowskiego,[],"Gdańsk, ...",47.00,2,3.0,NaN,unit,NaN,NaN
2,6800.0,Gdańsk Stare Miasto pomorskie,Mieszkanie Gdańsk Stare Miasto ul. Tobiasza,[],"Gdańsk, ...",100.40,4,1.0,2015.0,tenement,3,"winda, domofon"
3,5400.0,Gdańsk Oliwa pomorskie,Mieszkanie Gdańsk Oliwa ul. Grunwaldzka,[],"Gdańsk, ...",125.00,4,2.0,1960.0,tenement,2,NaN
4,2500.0,Gdańsk Piecki-Migowo pomorskie,Mieszkanie Gdańsk Piecki-Migowo ul. Myśliwska,[],"Gdańsk, ...",40.00,2,3.0,2019.0,apartment_building,4,winda


In [25]:
df_cleared["level"].isna().sum()

19

In [26]:
df_cleared['building_type'].unique()

array(['apartment_building', 'unit', 'tenement', nan], dtype=object)